In [1]:
import torch
import torch.nn as nn
class BasicUnit(nn.Module):
    """
        提取长短期特征
        输入通道数：32
        输出通道数：8
        参数：
        input_channels, output_channels,data
        返回：
        特征图 Tensor (N*C*L)
    """
    def __init__(self, input_channels, output_channels):
        super(BasicUnit, self).__init__()
        # 使用3x1的卷积核
        self.conv1 = nn.Conv1d(input_channels, output_channels, kernel_size=3, padding=1)  # 第一组卷积
        self.conv2 = nn.Conv1d(input_channels, output_channels, kernel_size=3, padding=1)  # 第二组卷积
        self.conv3 = nn.Conv1d(input_channels, output_channels, kernel_size=3, padding=1)  # 第三组卷积
        self.conv4 = nn.Conv1d(input_channels, output_channels, kernel_size=3, padding=1)  # 第四组卷积

    def forward(self, x):
        A = torch.tanh(self.conv1(x))  # A = tanh(x * w1 + b1)
        B = torch.sigmoid(self.conv2(x))  # B = σ(x * w2 + b2)
        C = torch.tanh(self.conv3(x))  # C = tanh(x * w3 + b3)
        D = torch.sigmoid(self.conv4(x))  # D = σ(x * w4 + b4)
        
        E = A * B + C * D  # E = A ⊗ B + C ⊗ D
        return E

class BrainAnalysisModule(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(BrainAnalysisModule, self).__init__()
        self.basic_unit1 = BasicUnit(input_channels, output_channels)  # 第一个基本单元
        self.basic_unit2 = BasicUnit(input_channels, output_channels)  # 第二个基本单元

    def forward(self, x):
        output1 = self.basic_unit1(x)  # 经过第一个基本单元
        output2 = self.basic_unit2(x)  # 经过第二个基本单元
        combined_output = output1 + output2  # 将两个输出相加
        return combined_output

# 示例使用
if __name__ == "__main__":
    # 假设输入数据形状为 (批大小, 通道数, 序列长度)
    input_data = torch.randn(40, 32, 300)  # 批大小为 40，通道数为 32，序列长度为 3000
    brain_module = BrainAnalysisModule(input_channels=32, output_channels=8)  # 创建模块实例
    output = brain_module(input_data)  # 前向传播
    print(output.shape)  # 打印输出形状


torch.Size([40, 8, 300])


In [2]:
class AnticipationModule:
    def __init__(self):
        pass
    
    def forward(self, predicted_modes):
        """
        计算重建预测值
        输入通道数：8
        输出通道数：1
        参数：
        predicted_modes: List[torch.Tensor] - 各个分解模式的预测值
        返回：
        torch.Tensor - 重建预测值
        """
        # 将所有模式的预测值求和
        reconstructed_prediction = sum(predicted_modes)
        return reconstructed_prediction

# 示例：使用 AnticipationModule
# if __name__ == "__main__":
#     # 假设有8个模式的预测值
#     predicted_modes = [torch.rand(1) for _ in range(8)]  # 生成随机预测值作为示例
# 
#     anticipation_module = AnticipationModule()
#     reconstructed_prediction = anticipation_module.forward(predicted_modes)
# 
#     print("重建预测值：", reconstructed_prediction.item())
#     print("预测值的维度：", reconstructed_prediction.size())  # 或者使用 reconstructed_prediction.shape

重建预测值： 3.883603096008301
预测值的维度： torch.Size([1])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
class ModelTrainer:
    def __init__(self, data_path, input_channels, output_channels, batch_size=40, num_epochs=200, initial_lr=0.01, min_lr=0.00001):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.data = self.load_data(data_path)
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.initial_lr = initial_lr
        self.min_lr = min_lr

        # 初始化模型
        self.brain_analysis_module = BrainAnalysisModule(input_channels, output_channels).to(self.device)
        self.anticipation_module = AnticipationModule().to(self.device)

        # 定义损失函数和优化器
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(list(self.brain_analysis_module.parameters()) + list(self.anticipation_module.parameters()), 
                                     lr=self.initial_lr, weight_decay=0.01)  # L2正则化

    def load_data(self, data_path):
        df = pd.read_csv(data_path)
        data = torch.tensor(df.values, dtype=torch.float32).to(self.device)
        return TensorDataset(data)

    def train(self):
        data_loader = DataLoader(self.data, batch_size=self.batch_size, shuffle=True)
        learning_rate = self.initial_lr
        decay_steps = self.num_epochs // 10

        for epoch in range(self.num_epochs):
            for inputs in data_loader:
                inputs = inputs[0].to(self.device)  # 获取输入数据并转移到 GPU

                # 前向传播
                y_hat = self.brain_analysis_module(inputs)
                final_output = self.anticipation_module(y_hat)

                # 计算损失
                loss = self.criterion(final_output, inputs)  # 假设目标是输入数据

                # 反向传播和优化
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            # 学习率衰减
            if epoch % decay_steps == 0 and learning_rate > self.min_lr:
                learning_rate *= 0.5  # 每10轮将学习率减半
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = learning_rate

            # 打印损失
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch + 1}/{self.num_epochs}], Loss: {loss.item():.4f}')

# 使用示例
data_path = 'dataset1.csv'  # 数据路径
input_channels = 32
output_channels = 8
trainer = ModelTrainer(data_path, input_channels, output_channels)
trainer.train()
